In [1]:
#import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pylab as pl
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import MiniBatchKMeans
from preprocess import *
from collections import Counter
import cv2
%load_ext autoreload
%autoreload 2

In [2]:
A = load_Dataset("../fonts-dataset/Scheherazade New/*.jpeg")
B= load_Dataset("../fonts-dataset/Lemonada/*.jpeg")
C= load_Dataset("../fonts-dataset/Marhey/*.jpeg")
D= load_Dataset("../fonts-dataset/IBM Plex Sans Arabic/*.jpeg")

400
400
400
400


In [3]:
A, A_TEST = train_test_split(A, test_size=0.2)
B, B_TEST = train_test_split(B, test_size=0.2)
C, C_TEST = train_test_split(C, test_size=0.2)
D, D_TEST = train_test_split(D, test_size=0.2)

In [4]:
A_PROCESSED = []
for img in A:
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img)
    A_PROCESSED.extend(words)
print("A: ", len(A_PROCESSED))
B_PROCESSED = []
for img in B:
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img)
    B_PROCESSED.extend(words)
print("B: ", len(B_PROCESSED))
C_PROCESSED = []
for img in C:
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img)
    C_PROCESSED.extend(words)
print("C: ", len(C_PROCESSED))
D_PROCESSED = []
for img in D:
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img)
    D_PROCESSED.extend(words)
print("D: ", len(D_PROCESSED))

A:  19849
B:  25270
C:  20689
D:  22368


In [5]:
def extract_sift_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return descriptors

In [6]:
def prepare_data(images):
    features = []
    for image in images:
        descriptors = extract_sift_features(image)
        if descriptors  is not None:
            features.append(descriptors)
    return features

In [7]:
def create_codebook(features, num_clusters, batch_size):
    # Create a MiniBatchKMeans clustering object
    kmeans = MiniBatchKMeans(n_clusters=num_clusters, batch_size=batch_size, random_state=0)

    # Fit the clustering model to the SIFT features
    kmeans.fit(features)

    return kmeans

In [8]:
def compute_bovw_representation(features, codebook):
    num_clusters = codebook.n_clusters
    bovw_representation = []

    for image_features in features:
        if len(image_features) > 0:
            # Assign each feature to a cluster
            image_features=image_features.reshape(-1,128)
            cluster_assignments = codebook.predict(image_features)

            # Create a histogram of cluster frequencies
            histogram = np.bincount(cluster_assignments, minlength=num_clusters)

            # Normalize the histogram
            histogram = histogram / np.sum(histogram)

            bovw_representation.append(histogram)
        else:
            # Handle cases where no features were detected
            bovw_representation.append(np.zeros(num_clusters))

    return bovw_representation

In [9]:
def train_knn_classifier(N,train_bovw_features):
    knn_classifier = NearestNeighbors(n_neighbors=N, algorithm='ball_tree', n_jobs=-1)

    # Fit the classifier on the training data
    knn_classifier.fit(train_bovw_features)

    return knn_classifier

In [30]:
def evaluate_classifier(classifier, test_bovw_features, train_label,test_label):
    #test_bovw_features=np.array(test_bovw_features).reshape(-1,1)
    # Find nearest neighbors for test data
    distances, indices = classifier.kneighbors(test_bovw_features)
    predictions = np.full(len(indices), -1)
    neighbor_labels = train_labels[indices]
    max_count_elements = np.array([np.argmax(np.bincount(array)) for array in neighbor_labels])
    #print(max_count_elements)
    # predictions = label_counts.argmax(axis=1)
    # empty=[]
    # for idx, arr in enumerate(indices):
    #     if (len(arr)==0):
    #         empty.append(True)
    #     else:
    #         empty.append(False)
    
   
    # empty=np.array(empty)
    # predictions[empty] = -1
    #empty=np.array(empty)
    #print(empty)        
    #print(predictions)
    #empty = np.any(indices == np.array([]), axis=1)

    # Filter out empty predictions
    # filtered_predictions = predictions[predictions != -1]

    return max_count_elements
    # for neighbors in indices:
    #     if len(neighbors) > 0:
    #         neighbor_labels = train_labels[neighbors]
    #         most_common_label = np.bincount(neighbor_labels).argmax()
    #         predictions.append(most_common_label)
    #     else:
    #         # Handle cases where no neighbors were found
    #         predictions.append(-1)

    # # Filter out test samples without predictions (-1)
    # filtered_predictions = []

    # for i, prediction in enumerate(predictions):
    #     if prediction != -1:
    #         filtered_predictions.append(prediction) 
    # return filtered_predictions
            
    

In [11]:
train_labels= np.array([0]*len(A_PROCESSED) + [1]*len(B_PROCESSED) + [2]*len(C_PROCESSED) + [3]*len(D_PROCESSED))

In [12]:
A_PROCESSED= prepare_data(A_PROCESSED)
B_PROCESSED= prepare_data(B_PROCESSED)
C_PROCESSED= prepare_data(C_PROCESSED)
D_PROCESSED= prepare_data(D_PROCESSED)

In [13]:
print("A: ", A_PROCESSED[10].shape)
print("B: ", B_PROCESSED[530].shape)
print("C: ", C_PROCESSED[320].shape)
print("D: ", D_PROCESSED[2034].shape)

A:  (15, 128)
B:  (20, 128)
C:  (31, 128)
D:  (28, 128)


In [14]:
combined_list = A_PROCESSED + B_PROCESSED + C_PROCESSED + D_PROCESSED
concatenated_train_sift_features = np.vstack(combined_list)
print(concatenated_train_sift_features.shape)

(2086587, 128)


In [15]:
num_clusters=100
batch_size=1000
codebook = create_codebook(concatenated_train_sift_features, 100, 1000)
print(codebook)

C:\Users\omara\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


MiniBatchKMeans(batch_size=1000, n_clusters=100, random_state=0)


In [16]:
train_bovw_features = compute_bovw_representation(combined_list, codebook)

In [17]:
A_PROCESSED=[]
B_PROCESSED=[]
C_PROCESSED=[]
D_PROCESSED=[]

In [18]:
knn_classifier = train_knn_classifier(5,train_bovw_features)


In [19]:
A_TPROCESSED = []
for img in A_TEST:
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img)
    A_TPROCESSED.append(words)
print("A: ", len(A_TPROCESSED))
B_TPROCESSED = []
for img in B_TEST:
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img)
    B_TPROCESSED.append(words)
print("B: ", len(B_TPROCESSED))
C_TPROCESSED = []
for img in C_TEST:
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img)
    C_TPROCESSED.append(words)
print("C: ", len(C_TPROCESSED))
D_TPROCESSED = []
for img in D_TEST:
    binr=threshold_image(img)
    binr=assure_white_bg(binr)
    img=denoise(img)
    words=get_words_grey(binr,img)
    D_TPROCESSED.append(words)
print("D: ", len(D_TPROCESSED))
 

A:  80
B:  80
C:  80
D:  80


In [31]:
counts = 0 
i =0
for words in A_TPROCESSED:
    DESC= prepare_data(words)
    test_bovw_features = compute_bovw_representation(DESC, codebook)
    preds= evaluate_classifier(knn_classifier, test_bovw_features, train_labels, 0)
    preds_count= Counter(preds)
    most_common_label = preds_count.most_common(1)[0][0]
    if most_common_label == 0:
        counts+=1
    print("iteration: ",i)
    print(preds)
    print(counts)
    i+=1
print("A: ", counts*100/len(A_TPROCESSED))
    

iteration:  0
[0 0 0 3 0 0 0 0 0 0 0 0 2 0 0 3 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0]
1
iteration:  1
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2
iteration:  2
[0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]
3
iteration:  3
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 3 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
4
iteration:  4
[0 0 0 0 0 0 3 0 0 0 0 0 1 0 0 0 0 0 0 2]
5
iteration:  5
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 3 0 0 0 0 0 0 0 0]
6
iteration:  6
[0 3 0 0 0 0 0 0 1 2 0 0 0 2 0 0 0 0 0 0 1 0 0 0 0 0 0 0 2 2 0 0 2 1 0 1 2
 3 0 0 0 0 0 2 0 0 0 0 0 0 0 1 0 3 3 0]
7
iteration:  7
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 2 0
 0 1 0 0 0 0 0 0 0 3 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 1 0]
8
iteration:  8
[0 0 0 1 0 0 0 0 0 2 0 0 0 0 0 0 

In [32]:
i =0
for words in B_TPROCESSED:
    DESC= prepare_data(words)
    test_bovw_features = compute_bovw_representation(DESC, codebook)
    preds= evaluate_classifier(knn_classifier, test_bovw_features, train_labels, 0)
    preds_count= Counter(preds)
    most_common_label = preds_count.most_common(1)[0][0]
    if most_common_label == 1:
        counts+=1
    print("iteration: ",i)
    print(preds)
    print(counts)
    i+=1
print("A: ", counts*100/(len(A_TPROCESSED)*2))

iteration:  0
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1]
80
iteration:  1
[1 1 1 2 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
81
iteration:  2
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 3 1 1 1 1 1 1 1 1 1 0 2 1 1 1
 1 1 0 1 3 1 1 3 1 3 1 1 0 1 1 1 1 3 1 1 3 1 3 1 1 1 3 0 1 3 3 1 1 1 1 1 1
 1 3 1 1 1 3 1 1 1 2 0 0 3 1 3 1 0 0 1 1 3 1 1 0 1 1 1 1 3 1 1 2 1 1 3 1 1
 0 1 1 1 1 1 1 0 1 3 1 3 2 1 1 3 1 1 1 1 1 3 3 1 1 1 1 1 1 0 1 1 1 3 1 1 1
 1]
82
iteration:  3
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1]
83
iteration:  4
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 3 1 3 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 3 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 3 3 1 

In [33]:
i = 0
for words in C_TPROCESSED:
    DESC= prepare_data(words)
    test_bovw_features = compute_bovw_representation(DESC, codebook)
    preds= evaluate_classifier(knn_classifier, test_bovw_features, train_labels, 0)
    preds_count= Counter(preds)
    most_common_label = preds_count.most_common(1)[0][0]
    if most_common_label == 2:
        counts+=1
    print("iteration: ",i)
    print(preds)
    print(counts)
    i+=1
print("A: ", counts*100/(3*len(A_TPROCESSED)))

iteration:  0
[2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 0 2 2 2 2 2
 2 3 2 2 2 2 0 2 2 2 2 2 2 2 2 3 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 0 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 1 2 2 2 2]
159
iteration:  1
[3 2 2 3 2 2 2 2 2 2 2 0 2 0 2 0 2 2 3]
160
iteration:  2
[2 2 2 1 2 2 2 0 3 2 2 2 0 2 0 2 2 2 2 2 2 2 0 0 2 3 2 2 2 2 2 2 0 1 2 2 2
 0 0 0 2 2 2 2 0 0 2 2 3 2 2 2 2 2 0 0 2 2 0 0 2 2 0 0 2 2 2 2 0 2 2 2 2 2
 2 1]
161
iteration:  3
[2 3 2 0 3 2 0 0 2 2 0 2 0 0 1 1 2 0 2 0 2 3 2 2 2 0 3 0 2 2 0 2 2 0 2 3 2
 2 3 2 0 0 0 0 0 0 2 2 2 3 2 0 3 0 0 2 0 3 2 0 0 1 0 2 0 2 0 2 2 2 2 2 3 0
 3 2 3 0 2 2 2 0 2 0 2 0 2 3 3 1 2 2 2 2 0 2 0 0 0 0 0 0 2 0 0 0 2 0 2 2 0
 0 3 0 2 2 3 2 2 0 0 0 0 2 0 3 2 2 3 0 2 2 2 0 0 2 2 2 0 1 2 3 0 0 0]
162
iteration:  4
[0 2 1 2 3 2 2 2 2 2 2 1 2 1 2 2 2 2 2 2 2 2 0 2 2 2 0 2 2 0]
163
iteration:  5
[2 2 3 2 2 2 2 2 0 2 2 0 0 3 2 2 2 2 2 2 2 2 2 2]
164
iteration:  6
[2 2 2 2 2 2 2 2 3 2 2 2 2 0 2 1 2 2 2 2 0 2 2 2 2 2 2 2 2 2 0 2 2 1 2 0 0
 2 2 2

In [34]:
i=0
for words in D_TPROCESSED:
    DESC= prepare_data(words)
    test_bovw_features = compute_bovw_representation(DESC, codebook)
    preds= evaluate_classifier(knn_classifier, test_bovw_features, train_labels, 0)
    preds_count= Counter(preds)
    most_common_label = preds_count.most_common(1)[0][0]
    if most_common_label == 3:
        counts+=1
    print("iteration: ",i)
    print(preds)
    print(counts)
    i+=1
print("A: ", counts*100/(4*len(A_TPROCESSED)))

iteration:  0
[3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 1 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 2 3 3 3
 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
236
iteration:  1
[3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3
 3]
237
iteration:  2
[1 3 3 3 1 3 3 3 0 3 1 3 3 3 3 3 3 3 0 3 3 3 1 3 1 3 1 3 3 3 3 0 3 3 3 3 3
 2 1 3 3 3 3 1 3 3 3 3 0 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3
 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 1 1 3 3 3 1 3 3 3 2 2 3 3 3 3 2 3 3 3 3
 3 3 1 3 2 3 3 3 3 3 3 3 3 1 3 3 3 1 2 3]
238
iteration:  3
[2 2 1 0 2 2 2 0 3 2 2 1 0 3 2 2 2 0 2 0 2 0 2 2 2 2 2 0 2 3]
238
iteration:  4
[3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 0 3 3
 3 3 3 3 3]
239
iteration:  5
[3 3 3 3 3]
240
iteration:  6
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 